In [1]:
# загружаем необходимые библиотеки, классы и функции
import numpy as np
import pandas as pd
import catboost
import sklearn
from sklearn.model_selection import (train_test_split,
                                     GridSearchCV)
from catboost import CatBoostClassifier, Pool 

In [2]:
print(np.__version__)
print(pd.__version__)
print(catboost.__version__)
print(sklearn.__version__)

1.21.5
1.4.3
1.0.6
1.0.2


In [3]:
# увеличиваем количество отображаемых столбцов
pd.set_option('display.max_columns', 300)

# загружаем исторический набор
data = pd.read_csv('Data/contest_train.csv')
data.head()

,ID,TARGET,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,FEATURE_9,FEATURE_10,FEATURE_11,FEATURE_12,FEATURE_13,FEATURE_14,FEATURE_15,FEATURE_16,FEATURE_17,FEATURE_18,FEATURE_19,FEATURE_20,FEATURE_21,FEATURE_22,FEATURE_23,FEATURE_24,FEATURE_25,FEATURE_26,FEATURE_27,FEATURE_28,FEATURE_29,FEATURE_30,FEATURE_31,FEATURE_32,FEATURE_33,FEATURE_34,FEATURE_35,FEATURE_36,FEATURE_37,FEATURE_38,FEATURE_39,FEATURE_40,FEATURE_41,FEATURE_42,FEATURE_43,FEATURE_44,FEATURE_45,FEATURE_46,FEATURE_47,FEATURE_48,FEATURE_49,FEATURE_50,FEATURE_51,FEATURE_52,FEATURE_53,FEATURE_54,FEATURE_55,FEATURE_56,FEATURE_57,FEATURE_58,FEATURE_59,FEATURE_60,FEATURE_61,FEATURE_62,FEATURE_63,FEATURE_64,FEATURE_65,FEATURE_66,FEATURE_67,FEATURE_68,FEATURE_69,FEATURE_70,FEATURE_71,FEATURE_72,FEATURE_73,FEATURE_74,FEATURE_75,FEATURE_76,FEATURE_77,FEATURE_78,FEATURE_79,FEATURE_80,FEATURE_81,FEATURE_82,FEATURE_83,FEATURE_84,FEATURE_85,FEATURE_86,FEATURE_87,FEATURE_88,FEATURE_89,FEATURE_90,FEATURE_91,FEATURE_92,FEATURE_93,FEATURE_94,FEATURE_95,FEATURE_96,FEATURE_97,FEATURE_98,FEATURE_99,FEATURE_100,FEATURE_101,FEATURE_102,FEATURE_103,FEATURE_104,FEATURE_105,FEATURE_106,FEATURE_107,FEATURE_108,FEATURE_109,FEATURE_110,FEATURE_111,FEATURE_112,FEATURE_113,FEATURE_114,FEATURE_115,FEATURE_116,FEATURE_117,FEATURE_118,FEATURE_119,FEATURE_120,FEATURE_121,FEATURE_122,FEATURE_123,FEATURE_124,FEATURE_125,FEATURE_126,FEATURE_127,FEATURE_128,FEATURE_129,FEATURE_130,FEATURE_131,FEATURE_132,FEATURE_133,FEATURE_134,FEATURE_135,FEATURE_136,FEATURE_137,FEATURE_138,FEATURE_139,FEATURE_140,FEATURE_141,FEATURE_142,FEATURE_143,FEATURE_144,FEATURE_145,FEATURE_146,FEATURE_147,FEATURE_148,FEATURE_149,FEATURE_150,FEATURE_151,FEATURE_152,FEATURE_153,FEATURE_154,FEATURE_155,FEATURE_156,FEATURE_157,FEATURE_158,FEATURE_159,FEATURE_160,FEATURE_161,FEATURE_162,FEATURE_163,FEATURE_164,FEATURE_165,FEATURE_166,FEATURE_167,FEATURE_168,FEATURE_169,FEATURE_170,FEATURE_171,FEATURE_172,FEATURE_173,FEATURE_174,FEATURE_175,FEATURE_176,FEATURE_177,FEATURE_178,FEATURE_179,FEATURE_180,FEATURE_181,FEATURE_182,FEATURE_183,FEATURE_184,FEATURE_185,FEATURE_186,FEATURE_187,FEATURE_188,FEATURE_189,FEATURE_190,FEATURE_191,FEATURE_192,FEATURE_193,FEATURE_194,FEATURE_195,FEATURE_196,FEATURE_197,FEATURE_198,FEATURE_199,FEATURE_200,FEATURE_201,FEATURE_202,FEATURE_203,FEATURE_204,FEATURE_205,FEATURE_206,FEATURE_207,FEATURE_208,FEATURE_209,FEATURE_210,FEATURE_211,FEATURE_212,FEATURE_213,FEATURE_214,FEATURE_215,FEATURE_216,FEATURE_217,FEATURE_218,FEATURE_219,FEATURE_220,FEATURE_221,FEATURE_222,FEATURE_223,FEATURE_224,FEATURE_225,FEATURE_226,FEATURE_227,FEATURE_228,FEATURE_229,FEATURE_230,FEATURE_231,FEATURE_232,FEATURE_233,FEATURE_234,FEATURE_235,FEATURE_236,FEATURE_237,FEATURE_238,FEATURE_239,FEATURE_240,FEATURE_241,FEATURE_242,FEATURE_243,FEATURE_244,FEATURE_245,FEATURE_246,FEATURE_247,FEATURE_248,FEATURE_249,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
0,0d1c880d23ff018,1,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-114.527812,-17.218055,0.0,0.0,0.0,25.257841,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.492541,0.0,3.770712,-2.165145,0.0,-11.508692,0.0,0.0,0.0,0.0,0.0,0.0,0.709990,-166.128241,16.497987,-109.953267,-10.257451,-35.403966,0.0,0.0,0.0,0.0,0.0,0.0,10.689090,0.868280,79.319996,57.563879,5.147706,219.773565,13.723706,79.067758,12.855037,-22.713935,-13.550348,-31.575653,64.609721,111.118840,104.421868,93.389622,33.815437,27.072520,60.966939,279.433996,242.857149,342.448089,180.439710,157.383234,12.0,0.0,436.976097,393.719775,438.089670,5.0,0.0,1.225806,20.945718,100.423397,-14.418549,11.459275,49.192649,58.212399,103.175453,30.562678,0.022050,12.683430,1.009054,-1.828095,0.040347,-11.521372,30.630117,-3.772832,-13.410730,17.525901,-0.033100,0.151530,8.523431,41.071996,0.367572,-0.267070,0.288801,-0.523745,48.307264,0.648305,0.123284,0.430367,6.378862,41.714275,1.401470,0.000000,0.125000,2.407001,8.846116,0.375000,5.0,3.0,0.0,0.0,0.

In [4]:
# увеличиваем количество отображаемых строк
pd.set_option('display.max_rows', 300)

# создаем пустые списки, в которые для каждой переменной 
# записываем количество уникальных значений, количество 
# пропусков, тип переменной
nunique_list = []
miss_list = []
type_list = []
for col in data.columns:
    nunique_list.append(data[col].nunique())
    miss_list.append(data[col].isnull().sum())
    type_list.append(data[col].dtypes)
    
# создаем датафрейм с информацией о количестве уникальных 
# значений, количестве пропусков, типе переменной
feat_labels = data.columns
summary = np.array([nunique_list, miss_list, type_list])
columns = ['nunique', 'missing', 'type']
results = pd.DataFrame(summary.T, 
                       index=feat_labels,
                       columns=columns)
results

,nunique,missing,type
ID,24521,0,object
TARGET,3,0,int64
FEATURE_0,160,0,float64
FEATURE_1,22,0,float64
FEATURE_2,2,0,float64
FEATURE_3,1,0,float64
FEATURE_4,2,0,float64
FEATURE_5,2,0,float64
FEATURE_6,2,0,float64
FEATURE_7,24521,0,float64


In [5]:
# удаляем переменную ID
data.drop('ID', axis=1, inplace=True)
# импутируем пропуски
data.fillna(-9999, inplace=True)
# удаляем константные признаки
constant_features = [
    feat for feat in data.columns if data[feat].nunique() == 1
]
data.drop(labels=constant_features, axis=1, inplace=True)

In [6]:
# посмотрим распределение классов зависимой переменной
data['TARGET'].value_counts(normalize=True)

0    0.708454
1    0.230415
2    0.061131
Name: TARGET, dtype: float64

In [7]:
# разбиваем набор на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('TARGET', axis=1), 
    data['TARGET'], 
    test_size=0.3,
    stratify=data['TARGET'],
    random_state=42)

In [8]:
# создаем экземпляр класса CatBoostClassifier
ctbst = CatBoostClassifier(loss_function='MultiClass',
                           iterations=200,
                           random_strength=0.15,
                           rsm=0.8,
                           random_seed=0,
                           logging_level='Silent')

In [9]:
# задаем сетку значений гиперпараметров
param_grid = {'auto_class_weights': ['None', 'Balanced'],
              'learning_rate': [0.1, 0.05]}

# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки, отключив запись метрик 
# для обучающих блоков перекрестной проверки в атрибут cv_results_
gs = GridSearchCV(ctbst, 
                  param_grid, 
                  cv=5, 
                  scoring='f1_macro', 
                  n_jobs=-1, 
                  return_train_score=False)

# выполняем поиск по сетке
gs.fit(X_train, y_train);

In [10]:
# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров: {}".format(gs.best_params_))
# смотрим наилучшее значение AUC
print("Наилучшее значение F1: {:.3f}".format(gs.best_score_))

Наилучшие значения гиперпараметров: {'auto_class_weights': 'Balanced', 'learning_rate': 0.1}
Наилучшее значение F1: 0.512


In [11]:
# смотрим результаты по всем комбинациям значений гиперпараметров
cv_results = pd.DataFrame(gs.cv_results_)[['mean_test_score',
                                           'param_auto_class_weights',
                                           'param_learning_rate']]
cv_results = cv_results.sort_values(
    by='mean_test_score', ascending=False).reset_index(drop=True)
cv_results

,mean_test_score,param_auto_class_weights,param_learning_rate
0,0.511888,Balanced,0.1
1,0.499382,Balanced,0.05
2,0.457361,None,0.1
3,0.429138,None,0.05


In [12]:
# создаем новый экземпляр класса CatBoostClassifier
ctbst2 = CatBoostClassifier(loss_function='MultiClass',
                            iterations=200,
                            rsm=0.8,
                            learning_rate=0.1,
                            auto_class_weights='Balanced',
                            random_strength=0.15,
                            random_seed=0,
                            logging_level='Silent')

In [13]:
# задаем новую сетку значений гиперпараметров
param_grid2 = {'depth': [6, 7, 8, 9]}

# создаем новый экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки, отключив запись метрик 
# для обучающих блоков перекрестной проверки в атрибут cv_results_
gs2 = GridSearchCV(ctbst2, 
                   param_grid2, 
                   cv=5, 
                   scoring='f1_macro', 
                   n_jobs=-1, 
                   return_train_score=False)
# выполняем поиск по сетке
gs2.fit(X_train, y_train);

In [14]:
# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров: {}".format(gs2.best_params_))
# смотрим наилучшее значение AUC
print("Наилучшее значение F1: {:.3f}".format(gs2.best_score_))

Наилучшие значения гиперпараметров: {'depth': 9}
Наилучшее значение F1: 0.536


In [15]:
# смотрим результаты
cv_results2 = pd.DataFrame(gs2.cv_results_)[['mean_test_score',
                                             'param_depth']]
cv_results2 = cv_results2.sort_values(
    by='mean_test_score', ascending=False).reset_index(drop=True)
cv_results2

,mean_test_score,param_depth
0,0.536066,9
1,0.527678,8
2,0.518201,7
3,0.511888,6


In [16]:
# загружаем наборы
train = pd.read_csv('Data/contest_train.csv')
test = pd.read_csv('Data/contest_test.csv')

# формируем массив меток и массив признаков
labels = train.pop('TARGET')
# сохраняем ID набора новых данных
test_id = test['ID']

# удаляем ID
train.drop('ID', axis=1, inplace=True)
test.drop('ID', axis=1, inplace=True)

# импутируем пропуски
train.fillna(-9999, inplace=True)
test.fillna(-9999, inplace=True)

# удаляем константные признаки
train.drop(labels=constant_features, axis=1, inplace=True)
test.drop(labels=constant_features, axis=1, inplace=True)

# формируем обучающий пул
train_pool = Pool(train, labels)

# создаем экземпляр класса CatBoostClassifier
clf_full = CatBoostClassifier(learning_rate=0.015,
                              loss_function='MultiClass',
                              depth=9,
                              rsm=0.8,
                              iterations=2000,
                              random_strength=0.15,
                              random_seed=0,
                              auto_class_weights='Balanced',
                              logging_level='Silent')

# обучаем модель
clf_full.fit(train_pool)
predictions = clf_full.predict(test)
# формируем посылку для Kaggle (2-е место на привате)
pd.DataFrame({'ID': test_id, 'Predicted': predictions.reshape(-1)}).to_csv(
    'subm_megafon.csv', index=False)